# Imports

In [1]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

import warnings
warnings.filterwarnings("ignore")

# 1. Dataset Preparation & Analysis

## 1.2 - Loading the dataset

In [2]:
data = pd.read_csv("../SOEN-471-Music-Recommender-System/data/data.csv") 
genre_data = pd.read_csv('../SOEN-471-Music-Recommender-System/data/data_by_genres.csv')
year_data = pd.read_csv('../SOEN-471-Music-Recommender-System/data/data_by_year.csv')

## 1.3 - Data Exploration

In [ ]:
# data.info()

In [ ]:
# data.describe()

In [ ]:
# genre_data.info()

In [ ]:
# year_data.info()

## 1.4 - Data Visualisation

In [ ]:
# from yellowbrick.target import FeatureCorrelation 

# feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration_ms','explicit','key','mode','year'] 

# X, y = data[feature_names], data['popularity'] 
# features = np.array(feature_names) # convert to numpy array for indexing
# visualizer = FeatureCorrelation(labels=features) # Instantiate the visualizer with the Covariance ranking algorithm

# plt.rcParams['figure.figsize']=(20,20) # set the size of the plot
# visualizer.fit(X, y) # Fit the data to the visualizer      
# visualizer.show() # Finalize and render the figure

In [ ]:
# # create a plotly figure
# sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']
# fig = px.line(year_data, x='year', y=sound_features, title='Trend of sound features over decades')
# fig.show() 

In [ ]:
# # create a plotly figure
# top10_genres = genre_data.nlargest(10, 'popularity')

# fig = px.bar(top10_genres, x='genres', y=['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence'], barmode='group',
#             title='Characteristics of top 10 genres') 
# fig.show()

## 1.7 - Content Based Recommendation

#### k-NN

###### The KNN algorithm is used to find the most similar songs to a given query song, based on their feature (such as loudness, tempo, and danceability) values. The data is preprocessed by rescaling values and dropping unnecessary features. The algorithm is trained on the dataset using a kd-tree implementation, which makes the nearest neighbor search more efficient.
###### The recommendation function takes as input a song ID and returns the top n (default is 5) most similar songs to that query song. The function uses the trained KNN model to find the nearest neighbors based on feature values and prints out the names and artists of the recommended songs.
###### This is a content-based recommendation system, as it only takes into account the features of the songs and not user preferences or behavior. The algorithm uses similarity in features to make recommendations.

In [ ]:
# data.head()

In [ ]:
# # Group by song name and count the number of unique artists
# grouped_by_song = data.groupby('name')['artists'].nunique()

# # Filter out unique song names and count the number of groups with more than 1 artist
# song_names_with_diff_artists = grouped_by_song[grouped_by_song > 1]
# count = sum(song_names_with_diff_artists)

# # Print the result
# print(f'There are {count} unique song names with different artists.')

# # Get an example of a song name in the list
# song_name = song_names_with_diff_artists.index[64]

# # Print all the rows with that song name
# print(f'All the rows with the song name "{song_name}":')
# print(data.loc[data['name'] == song_name])

In [3]:
# create a new dataframe and drop the columns that are not needed for the clustering 
df_for_kNN = data.drop(['artists','duration_ms','explicit','key','mode','id','popularity','release_date','year'],axis=1) 
df_for_kNN.head() # check the new dataframe

,valence,acousticness,danceability,energy,instrumentalness,liveness,loudness,name,speechiness,tempo
0,0.0594,0.982,0.279,0.211,0.878000,0.665,-20.096,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",0.0366,80.954
1,0.9630,0.732,0.819,0.341,0.000000,0.160,-12.441,Clancy Lowered the Boom,0.4150,60.936
2,0.0394,0.961,0.328,0.166,0.913000,0.101,-14.850,Gati Bali,0.0339,110.339
3,0.1650,0.967,0.275,0.309,0.000028,0.381,-9.316,Danny Boy,0.0354,100.109
4,0.2530,0.957,0.418,0.193,0.000002,0.229,-10.096,When Irish Eyes Are Smiling,0.0380,101.665


In [4]:
# Compute the minimum and maximum values for each column
for col in df_for_kNN.columns:
    min_val = df_for_kNN[col].min()
    max_val = df_for_kNN[col].max()
    print(f"{col}: min={min_val}, max={max_val}")

valence: min=0.0, max=1.0
acousticness: min=0.0, max=0.996
danceability: min=0.0, max=0.988
energy: min=0.0, max=1.0
instrumentalness: min=0.0, max=1.0
liveness: min=0.0, max=1.0
loudness: min=-60.0, max=3.855
name: min=!Que Vida! - Mono Version, max=텅 빈 마음 Empty Heart
speechiness: min=0.0, max=0.97
tempo: min=0.0, max=243.507


In [5]:
# rescale values and change ID to be primary key Spotify ID
df_for_kNN['loudness'] = df_for_kNN['loudness']+60 # since the minimum value is -60 and we want to make it 0
df_for_kNN['loudness'] = df_for_kNN['loudness']/63.855 # since the maximum value is now 63.855 and we want to make it 1
df_for_kNN['tempo'] = df_for_kNN['tempo']/244.091 # since the maximum value is 244.091 and we want to make it 1
df_for_kNN.index = df_for_kNN['name'] # change index to be the id
df_for_kNN = df_for_kNN.drop(['name'],axis=1) # remove id column
df_for_kNN.head() # check if everything is ok


,valence,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
name,,,,,,,,,
"Piano Concerto No. 3 in D Minor, Op. 30: III. Finale. Alla breve",0.0594,0.982,0.279,0.211,0.878000,0.665,0.624916,0.0366,0.331655
Clancy Lowered the Boom,0.9630,0.732,0.819,0.341,0.000000,0.160,0.744797,0.4150,0.249645
Gati Bali,0.0394,0.961,0.328,0.166,0.913000,0.101,0.707071,0.0339,0.452040
Danny Boy,0.1650,0.967,0.275,0.309,0.000028,0.381,0.793736,0.0354,0.410130
When Irish Eyes Are Smiling,0.2530,0.957,0.418,0.193,0.000002,0.229,0.781521,0.0380,0.416505


In [6]:
train, test = train_test_split(df_for_kNN, test_size=0.2, random_state=42) # split the data into train and test

In [7]:
# get song ids for the train and test sets
train_ids = train.index.tolist()
test_ids = test.index.tolist()

In [8]:
train.shape     # check the shape of the train set

(136522, 9)

In [9]:
test.shape      # check the shape of the test set

(34131, 9)

In [13]:
# create the kNN model using the train set
model = NearestNeighbors(n_neighbors=10, metric='cosine', algorithm='brute')
model.fit(train)


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [21]:
# find the k nearest neighbors for a random song in the test set and print the results
import random
random_song = random.choice(test_ids)
distances, indices = model.kneighbors(test.loc[random_song].values.reshape(1, -1), n_neighbors = 10)
print(f'For the song "{random_song}":')
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(random_song))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, train.index[indices.flatten()[i]], distances.flatten()[i]))
        


For the song "The Chain - Demo":
Recommendations for The Chain - Demo:

1: Forgive Myself, with distance of 0.0010062065543373633:
2: 一生中最愛 - 電影《雙城故事》歌曲, with distance of 0.00109929447909618:
3: Is It Over, with distance of 0.0011601648440753287:
4: Crowds, with distance of 0.0011915373706381516:
5: Who Would Have Thought, with distance of 0.001466207879999848:
6: Light Year, with distance of 0.0020519686564115514:
7: Enough To Be On Your Way, with distance of 0.0020809477009173794:
8: I Can Do All Things, with distance of 0.002083454048950939:
9: Overkill - Acoustic Version, with distance of 0.0021929860978140425:


In [ ]:
# # visualisation
# corr_matrix = df_for_kNN.corr() # Compute the correlation matrix
# sns.heatmap(corr_matrix, cmap='coolwarm', annot=True) # Plot the correlation matrix as a heatmap
# plt.title('Correlation Matrix of Song Features') 
# plt.show() 

In [ ]:
# model_knn = NearestNeighbors(algorithm='kd_tree',n_neighbors=6) # Create a KNN model
# mat_songs = csr_matrix(df_for_kNN.values) # Create a sparse matrix 
# model_knn.fit(mat_songs) # Fit the model to the sparse matrix
# Create the kNN model
# model = NearestNeighbors(n_neighbors=10, metric='cosine')
# model.fit(X_train)


In [ ]:
# import random

# # Choose a random song from the training set
# song_index = random.randint(0, X_train.get_shape()[0] - 1)
# song_features = X_train[song_index]
# song_id = train_ids[song_index]

# # Find the k nearest neighbors of the chosen song
# k = 10
# distances, indices = model.kneighbors([song_features.toarray()[0]], n_neighbors=k)

# # Print the recommended songs
# print(f"Because you listened to '{song_id}':")
# for i in range(1, k):
#     neighbor_song_id = train_ids[indices[0][i]]
#     print(f"{i}: '{neighbor_song_id}'")


In [ ]:
# def evaluate_knn_model(model, X_train, y_train, X_test, y_test, k):
#     model.fit(X_train)
#     distances, indices = model.kneighbors(X_test, n_neighbors=k)
#     y_pred = [y_train[indices[i][0]] for i in range(len(X_test))]
#     return f1_score(y_test, y_pred, average='weighted')


In [ ]:
# def recommend(id_song, model, number_of_recommendations=5): # Create a function to recommend songs
#     query = df_for_kNN.loc[id_song].to_numpy().reshape(1,-1) # Get the song features
#     print('Searching for recommendations...')
#     distances, indices = model.kneighbors(query,n_neighbors = number_of_recommendations) # Get the distances and indices of the nearest songs

#     # Loop through the indices and print the song names and artists of the nearest songs
#     for i, index in enumerate(indices[0]):
#      if index != id_song:
#         song_info = data[['name','artists']].loc[index]
#         print(f"{i+1}. {song_info['name']} by {song_info['artists']}")

In [ ]:
# name = input('Enter song title: ') # Get the song title from the user
# print('Song title search results: ') 
# print(data[['artists','name']].where(data['name'] == name).dropna()) # Print the search results

# song_ind = int(input('Enter the index value of the requested song: ')) # Get the index value of the song from the user since more than one song can have the same name
# id_song = data['id'].loc[song_ind] # Get the song id

# song = data['name'].loc[song_ind] # Get the song nam
# artists = data['artists'].loc[song_ind] # Get the song artists

# print('Song selected is ', song, 'by', artists) 

# nor = int(input('Enter number of recommendations: ')) # Get the number of recommendations (nor) from the user

# recommend(id_song, model_knn, nor) # Call the recommend function